In [1]:
import os
os.chdir("..")
!pwd

/Users/tung.dao/tung/mlopsvn/code/mlops-crash-course-code/training


In [2]:
import pandas as pd
import fastparquet
from pathlib import Path

## Load data

In [3]:
DATA_DIR = Path("./data")
TMP_DIR = Path("./tmp")
DATA_PATH = DATA_DIR / "driver_stats.parquet"
LABEL_PATH = DATA_DIR / "driver_orders.csv"
if not DATA_PATH.is_file():
    raise Exception("DATA_PATH not found")
if not LABEL_PATH.is_file():
    raise Exception("LABEL_PATH not found")

In [4]:
df_orig = pd.read_parquet(DATA_PATH, engine='fastparquet')
df_orig

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2021-07-13 11:00:00+00:00,1005,0.373837,0.154890,498,2021-07-28 11:08:04.802
1,2021-07-13 12:00:00+00:00,1005,0.571627,0.643958,656,2021-07-28 11:08:04.802
2,2021-07-13 13:00:00+00:00,1005,0.399909,0.993888,722,2021-07-28 11:08:04.802
3,2021-07-13 14:00:00+00:00,1005,0.967468,0.788458,424,2021-07-28 11:08:04.802
4,2021-07-13 15:00:00+00:00,1005,0.024679,0.956064,569,2021-07-28 11:08:04.802
...,...,...,...,...,...,...
1802,2021-07-28 09:00:00+00:00,1001,0.089418,0.311234,485,2021-07-28 11:08:04.802
1803,2021-07-28 10:00:00+00:00,1001,0.222534,0.927691,114,2021-07-28 11:08:04.802
1804,2021-04-12 07:00:00+00:00,1001,0.175219,0.761434,385,2021-07-28 11:08:04.802
902,2021-07-20 23:00:00+00:00,1003,0.025968,0.109748,55,2021-07-28 11:08:04.802


In [5]:
label_orig = pd.read_csv(LABEL_PATH, sep="\t")
label_orig

,event_timestamp,driver_id,trip_completed
0,2021-04-16 20:29:28+00:00,1001,1
1,2021-04-17 04:29:28+00:00,1002,0
2,2021-04-17 12:29:28+00:00,1003,0
3,2021-04-17 20:29:28+00:00,1001,1
4,2021-04-18 04:29:28+00:00,1002,0
5,2021-04-18 12:29:28+00:00,1003,0
6,2021-04-18 20:29:28+00:00,1001,1
7,2021-04-19 04:29:28+00:00,1002,0
8,2021-04-19 12:29:28+00:00,1003,0
9,2021-04-19 20:29:28+00:00,1004,1


### Format timestamp

In [6]:
label_orig["event_timestamp"] = pd.to_datetime(label_orig["event_timestamp"])
label_orig

,event_timestamp,driver_id,trip_completed
0,2021-04-16 20:29:28+00:00,1001,1
1,2021-04-17 04:29:28+00:00,1002,0
2,2021-04-17 12:29:28+00:00,1003,0
3,2021-04-17 20:29:28+00:00,1001,1
4,2021-04-18 04:29:28+00:00,1002,0
5,2021-04-18 12:29:28+00:00,1003,0
6,2021-04-18 20:29:28+00:00,1001,1
7,2021-04-19 04:29:28+00:00,1002,0
8,2021-04-19 12:29:28+00:00,1003,0
9,2021-04-19 20:29:28+00:00,1004,1


In [7]:
target = "trip_completed"

## Explore data

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1807 entries, 0 to 902
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   datetime         1807 non-null   datetime64[ns, UTC]
 1   driver_id        1807 non-null   int64              
 2   conv_rate        1807 non-null   float64            
 3   acc_rate         1807 non-null   float64            
 4   avg_daily_trips  1807 non-null   int64              
 5   created          1807 non-null   datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(2), int64(2)
memory usage: 98.8 KB


In [10]:
df_orig.describe()

,driver_id,conv_rate,acc_rate,avg_daily_trips
count,1807.000000,1807.000000,1807.000000,1807.000000
mean,1003.000000,0.488267,0.505205,500.871057
std,1.413822,0.291862,0.291230,293.412315
min,1001.000000,0.000482,0.000542,0.000000
25%,1002.000000,0.238879,0.251682,236.000000
50%,1003.000000,0.491606,0.507843,506.000000
75%,1004.000000,0.732576,0.770225,754.000000
max,1005.000000,0.998767,0.999445,998.000000


In [11]:
label_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   event_timestamp  10 non-null     datetime64[ns, UTC]
 1   driver_id        10 non-null     int64              
 2   trip_completed   10 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(2)
memory usage: 368.0 bytes


In [12]:
label_orig.describe()

,driver_id,trip_completed
count,10.000000,10.000000
mean,1002.200000,0.400000
std,1.032796,0.516398
min,1001.000000,0.000000
25%,1001.250000,0.000000
50%,1002.000000,0.000000
75%,1003.000000,1.000000
max,1004.000000,1.000000


## Training

### Split data

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
training_df = df_orig.merge(label_orig, left_on="driver_id", right_on="driver_id")
training_df = training_df[training_df.columns. \
    drop("datetime"). \
    drop("driver_id"). \
    drop("created"). \
    drop("event_timestamp")]
training_df

,conv_rate,acc_rate,avg_daily_trips,trip_completed
0,0.247582,0.795497,772,1
1,0.875381,0.025149,203,1
2,0.977574,0.408694,158,1
3,0.391113,0.092807,731,1
4,0.611149,0.144825,468,1
...,...,...,...,...
3611,0.222534,0.927691,114,1
3612,0.222534,0.927691,114,1
3613,0.175219,0.761434,385,1
3614,0.175219,0.761434,385,1


In [15]:
train_X = training_df[training_df.columns.drop(target)]
train_Y = training_df.loc[:, target]

In [16]:
reg = LinearRegression()
reg.fit(train_X[sorted(train_X)], train_Y)

LinearRegression()

### Export model

In [17]:
from joblib import dump

In [18]:
MODEL_DIR = Path("./model")

In [19]:
dump(reg, MODEL_DIR / "driver_model.bin")

['model/driver_model.bin']